<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=973841b987ad20d0b3512bf3b85222f8830da9bf529c8749d95c8bb90d31c5e7
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7


Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-23 14:07:34
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16


-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.41 C
-------------------
Today PnL: -51.59 K (-0.36%)
Current PnL: -18.64 L (-12.54%)
CY Booked + Current PnL: -7.20 L (-4.84%)
-------------------
Total profit:  1.64 L
Total loss:  -20.28 L
-------------------
Total Booked + Current PnL: 19.78 L (16.26%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.12%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 84.50 L (59.95%)
Deployed:  1.22 C
Current:  1.41 C
CAGR/XIRR %: 9.01%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.77,61.0,H-MC,3.63,196490.0,31820.0,11514.0,-1.60,19.32,5.86,26.31,89.0,2.76,1.39,28.85,X40N,NTT,AC
77,TTKPRESTIG,770.00,97.65,42.0,M-SC,3.09,85202.0,-15575.0,15669.0,-0.97,-15.45,18.39,0.09,245.0,-0.99,0.60,11.94,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-43.66,64.0,H-MC,9.28,111671.0,11167.0,17946.0,-0.11,11.11,16.07,28.96,79.0,0.62,0.79,48.85,MH,ATH,METALS
57,RELIANCE,1533.00,-13.29,50.0,H-LC,4.60,217448.0,7082.0,21701.0,0.24,3.37,9.98,13.68,37.0,0.33,1.54,20.45,XY25,NTT,REFINERIES
84,VOLTAS,1530.00,-1.64,47.0,H-MC,2.85,206400.0,14658.0,23096.0,-0.01,7.64,11.19,19.69,99.0,0.63,1.46,14.96,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.0,-41.65,53.0,L-SC,42.55,86016.0,-27533.0,67583.0,4.28,-24.25,78.57,35.27,268.0,-0.41,0.61,110.53,XR,NTT,HOTELS
7,ATULAUTO,844.0,3829.73,59.0,M-SC,5.34,122437.0,-24931.0,76744.0,3.36,-16.92,62.68,35.16,236.0,-0.32,0.87,25.06,XY24,NTT,AUTO
39,INDUSINDBK,1800.0,250.84,49.0,L-MC,12.24,43092.0,-34114.0,59510.0,2.96,-44.19,138.10,32.89,258.0,-0.57,0.31,18.67,XR,NTT,BANKS
66,SIS,528.0,2088.08,39.0,H-SC,4.85,88306.0,-22726.0,46334.0,1.61,-20.47,52.47,21.26,156.0,-0.49,0.63,19.31,OX40N,NTT,MISC
79,UNIONBANK,163.0,-11.63,66.0,M-LC,7.04,160237.0,19397.0,25093.0,1.53,13.77,15.66,31.59,66.0,0.77,1.14,43.91,XY24,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-65.02,51.0,H-MC,3.79,249016.0,-52735.0,214154.0,-3.82,-17.48,86.00,53.50,116.0,-0.25,1.77,10.60,XY24,NTT,FMCG
45,KANSAINER,340.00,-67.97,51.0,H-SC,0.84,221400.0,-48267.0,84597.0,-2.77,-17.90,38.21,13.47,138.0,-0.57,1.57,11.98,XY24,NTT,PAINTS
6,ASIANTILES,137.00,6905.56,57.0,L-SC,5.21,76842.0,-16968.0,93586.0,-2.28,-18.09,121.79,81.67,269.0,-0.18,0.55,48.81,XR,NTT,CERAMICS
61,SAMMAANCAP,326.00,-171.59,59.0,M-SC,4.45,82920.0,-19290.0,112680.0,-2.10,-18.87,135.89,91.37,208.0,-0.17,0.59,34.61,XY25,NTT,FINANCE
37,INDIAMART,4850.92,-54.45,28.0,H-SC,11.00,127200.0,3864.0,129897.0,-2.01,3.13,102.12,108.45,119.0,0.03,0.90,26.19,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1233.33,76.0,M-SC,8.10,181289.0,146.0,45322.0,-1.51,0.08,25.00,25.10,235.0,0.00,1.29,45.28,XY24,NTT,HEALTHCARE
35,ICICIPRULI,790.00,-20.88,45.0,H-MC,2.07,136222.0,434.0,42324.0,-1.11,0.32,31.07,31.48,107.0,0.01,0.97,12.48,X40,ATH,INSURANCE
23,DABUR,735.00,-7.80,38.0,H-MC,6.89,249751.0,1683.0,106719.0,-1.36,0.68,42.73,43.70,102.0,0.02,1.77,15.22,XY24,BTT,FMCG
29,HAVELLS,2069.16,-0.94,52.0,H-MC,2.03,245731.0,-2000.0,77061.0,-0.39,-0.81,31.36,30.30,92.0,-0.03,1.74,12.38,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.28,53.0,H-MC,3.79,201945.0,1125.0,67652.0,-0.52,0.56,33.50,34.25,80.0,0.02,1.43,5.79,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,91.37,49.0,H-SC,9.29,120879.0,-21600.0,41123.0,-0.00,-15.16,34.02,13.70,163.0,-0.53,0.86,42.67,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,97.65,42.0,M-SC,3.09,85202.0,-15575.0,15669.0,-0.97,-15.45,18.39,0.09,245.0,-0.99,0.60,11.94,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,51.0,H-SC,0.84,221400.0,-48267.0,84597.0,-2.77,-17.90,38.21,13.47,138.0,-0.57,1.57,11.98,XY24,NTT,PAINTS
18,CERA,9475.0,-21.96,37.0,H-SC,2.11,142347.0,-33556.0,75572.0,-1.79,-19.08,53.09,23.89,149.0,-0.44,1.01,22.94,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-22.20,43.0,H-MC,7.09,105279.0,-26556.0,68779.0,-1.18,-20.14,65.33,32.03,98.0,-0.39,0.75,17.92,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1233.33,76.0,M-SC,8.10,181289.0,146.0,45322.0,-1.51,0.08,25.00,25.10,235.0,0.00,1.29,45.28,XY24,NTT,HEALTHCARE
60,SAIL,228.00,49.96,71.0,M-MC,8.97,236465.0,11503.0,156161.0,1.35,5.11,66.04,74.53,192.0,0.07,1.68,39.02,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.77,61.0,H-MC,3.63,196490.0,31820.0,11514.0,-1.60,19.32,5.86,26.31,89.0,2.76,1.39,28.85,X40N,NTT,AC
17,CAMS,5211.76,-1.62,57.0,H-SC,2.62,111888.0,9884.0,34036.0,-0.45,9.69,30.42,43.06,122.0,0.29,0.79,28.46,X40N,ATH,MISC
84,VOLTAS,1530.00,-1.64,47.0,H-MC,2.85,206400.0,14658.0,23096.0,-0.01,7.64,11.19,19.69,99.0,0.63,1.46,14.96,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-43.66,64.0,H-MC,9.28,111671.0,11167.0,17946.0,-0.11,11.11,16.07,28.96,79.0,0.62,0.79,48.85,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-41.30,47.0,M-SC,1.13,99322.0,7824.0,70926.0,-0.37,8.55,71.41,86.07,223.0,0.11,0.70,44.23,XR,NTT,DURABLES
11,BANKINDIA,190.00,-27.95,69.0,H-MC,7.01,186652.0,6844.0,105570.0,0.69,3.81,56.56,62.52,88.0,0.06,1.32,39.19,XR,NTT,BANKS
86,WIPRO,420.00,-11.07,51.0,M-LC,5.66,155930.0,4985.0,104473.0,0.47,3.30,67.00,72.51,53.0,0.05,1.11,9.52,XR,NTT,IT
37,INDIAMART,4850.92,-54.45,28.0,H-SC,11.00,127200.0,3864.0,129897.0,-2.01,3.13,102.12,108.45,119.0,0.03,0.90,26.19,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-18.54,36.0,H-LC,7.17,293544.0,-22298.0,134972.0,-0.83,-7.06,45.98,35.67,5.0,-0.17,2.08,5.82,X40N,ATH,FMCG
31,HINDUNILVR,2922.00,-9.85,39.0,H-LC,10.29,252990.0,-4073.0,39213.0,-1.64,-1.58,15.50,13.67,24.0,-0.10,1.80,18.50,XY25,NTT,FMCG
30,HCLTECH,1943.91,-1.08,41.0,H-LC,9.90,225593.0,-16331.0,79589.0,-0.21,-6.75,35.28,26.15,8.0,-0.21,1.60,6.61,X40,ATH,IT
5,ASIANPAINT,3465.66,-14.24,41.0,H-LC,7.63,216357.0,-35411.0,88620.0,-0.97,-14.06,40.96,21.13,27.0,-0.40,1.54,16.38,X40,ATH,PAINTS
42,ITC,452.00,-38.77,42.0,H-LC,2.04,197678.0,-2460.0,23346.0,-0.66,-1.23,11.81,10.44,4.0,-0.11,1.40,4.27,X40,NTT,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-4.05,51.0,H-LC,1.49,238072.0,-8079.0,87277.0,0.58,-3.28,36.66,32.18,16.0,-0.09,1.69,31.58,X200,ATH,IT
42,ITC,452.0,-38.77,42.0,H-LC,2.04,197678.0,-2460.0,23346.0,-0.66,-1.23,11.81,10.44,4.0,-0.11,1.40,4.27,X40,NTT,FMCG
65,SIEMENS,4671.5,0.81,54.0,H-LC,3.33,162115.0,-23980.0,71460.0,-0.04,-12.89,44.08,25.51,15.0,-0.34,1.15,19.47,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-13.29,50.0,H-LC,4.60,217448.0,7082.0,21701.0,0.24,3.37,9.98,13.68,37.0,0.33,1.54,20.45,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-9.62,50.0,H-LC,4.61,278062.0,12636.0,44156.0,-0.52,4.76,15.88,21.40,11.0,0.29,1.97,12.92,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.40,45.0,H-LC,12.15,289135.0,-56821.0,131672.0,0.07,-16.42,45.54,21.64,1.0,-0.43,2.05,2.43,X40,ATH,IT
42,ITC,452.00,-38.77,42.0,H-LC,2.04,197678.0,-2460.0,23346.0,-0.66,-1.23,11.81,10.44,4.0,-0.11,1.40,4.27,X40,NTT,FMCG
80,UNITDSPR,1644.00,-12.23,55.0,H-LC,7.50,230119.0,-4025.0,52651.0,-0.15,-1.72,22.88,20.77,86.0,-0.08,1.63,5.08,X40N,NTT,BREWERIES
83,VBL,671.64,-18.54,36.0,H-LC,7.17,293544.0,-22298.0,134972.0,-0.83,-7.06,45.98,35.67,5.0,-0.17,2.08,5.82,X40N,ATH,FMCG
30,HCLTECH,1943.91,-1.08,41.0,H-LC,9.90,225593.0,-16331.0,79589.0,-0.21,-6.75,35.28,26.15,8.0,-0.21,1.60,6.61,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,0.81,54.0,H-LC,3.33,162115.0,-23980.0,71460.0,-0.04,-12.89,44.08,25.51,15.0,-0.34,1.15,19.47,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,42.0,H-LC,2.04,197678.0,-2460.0,23346.0,-0.66,-1.23,11.81,10.44,4.0,-0.11,1.40,4.27,X40,NTT,FMCG
19,CIPLA,1795.00,-21.49,42.0,H-LC,5.32,209720.0,5220.0,36198.0,-0.74,2.55,17.26,20.25,10.0,0.14,1.49,11.51,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-14.24,41.0,H-LC,7.63,216357.0,-35411.0,88620.0,-0.97,-14.06,40.96,21.13,27.0,-0.40,1.54,16.38,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.29,50.0,H-LC,4.60,217448.0,7082.0,21701.0,0.24,3.37,9.98,13.68,37.0,0.33,1.54,20.45,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-27.40,45.0,H-LC,12.15,289135.0,-56821.0,131672.0,0.07,-16.42,45.54,21.64,1.0,-0.43,2.05,2.43,X40,ATH,IT
40,INFY,2275.00,-17.80,50.0,H-LC,8.15,320757.0,7869.0,163811.0,0.43,2.51,51.07,54.87,3.0,0.05,2.28,9.33,X40,BTT,IT
42,ITC,452.00,-38.77,42.0,H-LC,2.04,197678.0,-2460.0,23346.0,-0.66,-1.23,11.81,10.44,4.0,-0.11,1.40,4.27,X40,NTT,FMCG
83,VBL,671.64,-18.54,36.0,H-LC,7.17,293544.0,-22298.0,134972.0,-0.83,-7.06,45.98,35.67,5.0,-0.17,2.08,5.82,X40N,ATH,FMCG
1,ABB,7934.00,-38.99,55.0,H-LC,9.02,255120.0,-6499.0,125723.0,-0.89,-2.48,49.28,45.57,7.0,-0.05,1.81,8.56,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6905.56,57.0,L-SC,5.21,76842.0,-16968.0,93586.0,-2.28,-18.09,121.79,81.67,269.0,-0.18,0.55,48.81,XR,NTT,CERAMICS
61,SAMMAANCAP,326.00,-171.59,59.0,M-SC,4.45,82920.0,-19290.0,112680.0,-2.10,-18.87,135.89,91.37,208.0,-0.17,0.59,34.61,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-41.65,53.0,L-SC,42.55,86016.0,-27533.0,67583.0,4.28,-24.25,78.57,35.27,268.0,-0.41,0.61,110.53,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-6.52,42.0,H-SC,12.05,87075.0,-13888.0,108417.0,-0.15,-13.76,124.51,93.63,148.0,-0.13,0.62,29.70,SR,ATH,CHEMICALS
49,MASFIN,398.61,-17.34,49.0,H-SC,12.67,93855.0,-4125.0,25726.0,-0.64,-4.21,27.41,22.05,152.0,-0.16,0.67,36.43,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1233.33,76.0,M-SC,8.10,181289.0,146.0,45322.0,-1.51,0.08,25.00,25.10,235.0,0.00,1.29,45.28,XY24,NTT,HEALTHCARE
7,ATULAUTO,844.00,3829.73,59.0,M-SC,5.34,122437.0,-24931.0,76744.0,3.36,-16.92,62.68,35.16,236.0,-0.32,0.87,25.06,XY24,NTT,AUTO
71,TANLA,1963.11,-15.40,71.0,H-SC,14.41,226532.0,-44229.0,374186.0,0.70,-16.34,165.18,121.86,133.0,-0.12,1.61,77.21,AR,ATH,IT
81,VAIBHAVGBL,521.00,61.98,63.0,H-SC,6.02,140549.0,-42226.0,153297.0,-0.60,-23.10,109.07,60.77,125.0,-0.28,1.00,25.88,XR,NTT,JEWELLERY
61,SAMMAANCAP,326.00,-171.59,59.0,M-SC,4.45,82920.0,-19290.0,112680.0,-2.10,-18.87,135.89,91.37,208.0,-0.17,0.59,34.61,XY25,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.25
1,25,43.66
2,50,74.77


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.00
LC    32.46
MC    23.53
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.35
X40      14.73
X40N     11.38
XR       11.29
XY25     10.05
AR        8.39
OX40N     7.81
X200      1.69
MH        1.65
X5K       1.42
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.37
H-LC    25.94
H-MC    20.56
M-SC    13.11
M-LC     5.48
M-MC     2.66
L-SC     1.52
L-LC     1.04
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.87,-4.94,39.57
IT,13.00,-14.85,77.47
FINANCE,10.06,-15.75,66.25
MISC,7.04,-15.66,79.97
BANKS,6.12,-15.94,76.93
PAINTS,5.61,-16.88,34.30
ELECTRICAL,5.48,-6.56,43.95
INSURANCE,3.71,-5.99,45.61
AC,3.54,3.31,12.88


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3104336.0,22
XR,1319580.0,14
AR,1148536.0,9
X40,773390.0,10
X40N,599297.0,9
OX40N,546774.0,10
XY25,422786.0,7
SR,249059.0,2
X5K,117770.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3656212.0,29
M-SC,1396141.0,17
H-MC,1206222.0,15
H-LC,1164198.0,15
M-LC,378952.0,4
M-MC,303639.0,2
L-SC,247339.0,3
L-MC,59510.0,1
L-LC,38135.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1226209.0      6
M-SC       XY24       796744.0      7
H-SC       AR         786370.0      5
           XR         773347.0      7
H-MC       XY24       557226.0      4
H-LC       X40        487038.0      5
M-MC       XY24       303639.0      2
H-SC       X40N       299677.0      4
           OX40N      257953.0      4
           SR         249059.0      2
H-LC       X40N       223821.0      3
H-MC       X40        214001.0      4
H-LC       AR         197183.0      2
H-MC       XY25       166901.0      2
M-SC       AR         164983.0      2
L-SC       XR         161169.0      2
M-LC       XY24       156709.0      2
M-SC       OX40N      133872.0      4
M-LC       X5K        117770.0      1
M-SC       XR         115511.0      2
           XY25       112680.0      1
H-MC       XR         105570.0      1
H-LC       XY25       105070.0      3
M-LC       XR         104473.0      1
H-LC       X200        87277.0      1
L-SC       OX40N       86170.0      1
H-MC       X40N        75799.0      2
M-SC       X40         72351.0      1
H-MC       OX40N       68779.0      1
H-LC       XY24        63809.0      1
H-SC       MH          63597.0      1
L-MC       XR          59510.0      1
L-LC       XY25        38135.0      1
H-MC       MH          17946.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 40.0 seconds
